In [1]:
import os

import streamlit as st
from dotenv import load_dotenv

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [3]:
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate

In [4]:
# load_dotenv(".env")
# api_key = os.environ.get("OPENAI_API_KEY")

In [5]:
vector_store_persist_directory = "./vector-store/asu_ai_db_tr"
original_document_path = "./data/yeni_sss.txt"

In [6]:
# vector_store_persist_directory = "./vector-store/asu_ai_db_tr_SSS"
# original_document_path = "./data/SSS.txt"

In [7]:
embedding_function = OpenAIEmbeddings()

In [8]:
# if os.path.exists(vector_store_persist_directory):
#     # Load documents
#     loader = TextLoader(original_document_path, encoding="UTF-8")
#     documents = loader.load()
#     text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=200)

#     # Split and persist documents
#     docs = text_splitter.split_documents(documents)
#     db = Chroma.from_documents(
#         docs, embedding_function, persist_directory=vector_store_persist_directory
#     )
#     db.persist()
# else:
#     db = Chroma(persist_directory=vector_store_persist_directory, embedding_function=embedding_function)

In [9]:
db = Chroma(persist_directory=vector_store_persist_directory, embedding_function=embedding_function)

In [10]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [11]:
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(),
    llm=llm,
)

In [12]:
def answer_my_question(question):
    unique_docs = retriever_from_llm.get_relevant_documents(query=question)

    human_message_template = "{question} sorusuna aşağıda verilen bağlamdaki bilgilere ugun şekilde cevap ver.\n{baglam}." 
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_message_template)
    system_message_template = "Cevapta sadece sorunun en kısa cevabını döndür. Soru yeterince açık değilse soru alternatifleri vererek tekrar sorulmasını iste."
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_message_template)

    prompt = ChatPromptTemplate.from_messages([human_message_prompt,system_message_prompt])
    formatted_question = prompt.format_prompt(question=question, baglam=unique_docs).to_messages()
    result = llm(formatted_question)
    return result.content

In [13]:
# answer_my_question('Lisansüstü programlara ücret ödeyecek miyim?')

In [14]:
questions = [
                # "Lisansüstü programlara ücret ödeyecek miyim? Doktora, tezsiz yüksek lisans ve yüksek lisans için ücretler kaç TL?",
                # "Lisansüstü programlara ücret ödeyecek miyim?",
                # "hangi lisansüstü programlar ücretsizdir?",
                # "hangi lisansüstü programlar ücretlidir?",
                # "tezsiz yüksek lisans programlara ücret ödeyecek miyim?",
                # "şehit ve gazi yakınları için ücretli mi?",
                # "burs başvurusu yapabilir miyim?",
                # "nasıl burs başvurusu yapabilirim?",
                # "nasıl yatay geçiş yapabilirim",
                # "Lisansüstü programlarda kaç ders alırım",
                # "Tezsiz Yüksek Lisansta kaç ders alırım",
                # "Tezli Yüksek Lisansta kaç ders alırım",
                # "doktorada kaç ders alırım",
                "yüksek lisans tezi nasıl yazılır?",
                "yüksek lisans tez savunması nasıl yapılır?",
                "tez savunmasında başarısız öğrenciler ne yapmalı?",
                "tez nasıl yazılır?",    
            ]

In [15]:
qnas = []

In [16]:
for question in questions:
    qnas.append(
        {
            "question":question,
             "answer": answer_my_question(question)
        }
    )
    print(question)

Tezsiz Yüksek Lisansta kaç ders alırım
Tezli Yüksek Lisansta kaç ders alırım
doktorada kaç ders alırım


In [17]:
for item in qnas:
    print(f"## Soru: {item['question']}")
    print(f"Cevap: {item['answer']}")
    print("")

## Soru: Tezsiz Yüksek Lisansta kaç ders alırım
Cevap: Tezsiz Yüksek Lisans programında bir yarıyıl içerisinde en az 3, en fazla 7 ders alabilirsiniz.

## Soru: Tezli Yüksek Lisansta kaç ders alırım
Cevap: Tezli Yüksek Lisans Programında bir yarıyılda en az 2, en fazla 5 ders (seçmeli veya zorunlu) + Uzmanlık Alan Dersi alabilirsiniz.

## Soru: doktorada kaç ders alırım
Cevap: Doktora programında bir dönemde en az 2, en fazla 5 ders alabilirsiniz. Uzmanlık alan dersi de dahildir.



In [18]:
# print(answer_my_question("Tezli Yüksek Lisansta kaç ders alırım",))